# Ver 1 using char-cnn model

In [15]:
from theano.sandbox import cuda
%matplotlib inline
from __future__ import division, print_function
import operator

import numpy as np
from keras.layers import TimeDistributed, Activation
from numpy.random import choice
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2, activity_l2, l1, activity_l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.utils.layer_utils import layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from numpy.random import choice
import time
import codecs

In [2]:
# use entire text
text = open('./data/lgqm.txt').read()
text = unicode(text, "utf-8")
print('length of text: ', len(text))
print(text[:100])
charset = set(text)
char_freq_map = {c: 0 for c in charset}
for c in text:
    char_freq_map[c] += 1
char_freq_list = sorted(char_freq_map.items(), key=operator.itemgetter(1))
char_freq_list.reverse()
charlist = [pair[0] for pair in char_freq_list]
print('size of charset', len(charlist))
for i in range(40):
    print(charlist[i])
char2idx = dict((c, i) for i, c in enumerate(charlist))
text_in_idx = [char2idx[c] for c in text]
#text_in_idx[:24]
print(''.join(charlist[i] for i in text_in_idx[:24]))
maxlen = 48
sentences = []
next_chars = []
for i in range(0, len(text_in_idx) - maxlen):
    sentences.append(text_in_idx[i: i + maxlen])
    next_chars.append(text_in_idx[i+1: i+maxlen+1])
print('nb sequences:', len(sentences))
sentences = np.array(sentences)
next_chars = np.array(next_chars)
sentences.shape, next_chars.shape
print('xxx done')

length of text:  9215340
《临高启明》吹牛者

严正声明：本书为丫丫小说网(www.shuyaya.com)的用户上传至其在本站的存储空间，本站只提供TXT全集电子书存储服务以及免费下载服务，以下作品内容之版权与本站无任何
size of charset 5688
 
，
的



。
是
了
一
不
“
”
这
人
有
在
他
来
个
大
上
就
得
到
要
们
和
说
里
―
也
着
子
下
还
出
过
地
会
道
《临高启明》吹牛者

严正声明：本书为丫丫小
nb sequences: 9215292
xxx done


In [7]:
n_fac = 100
batch_size = 64
n_hidden = 128
vocab_size = len(charlist)
model = Sequential([
        Embedding(vocab_size, n_fac, input_length=maxlen, batch_input_shape=(batch_size, maxlen)),
        BatchNormalization(),
        LSTM(n_hidden, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        LSTM(n_hidden, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
             consume_less='gpu'),
        Dropout(0.2),
        TimeDistributed(Dense(n_hidden, activation='relu')),
        Dropout(0.2),
        TimeDistributed(Dense(vocab_size, activation='softmax')),
])
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())
model.summary()
model.optimizer.lr=0.01

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (64, 48, 100)         568800      embedding_input_2[0][0]          
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormal(64, 48, 100)         200         embedding_2[0][0]                
____________________________________________________________________________________________________
lstm_3 (LSTM)                    (64, 48, 128)         117248      batchnormalization_2[0][0]       
____________________________________________________________________________________________________
dropout_4 (Dropout)              (64, 48, 128)         0           lstm_3[0][0]                     
___________________________________________________________________________________________

In [19]:
def run_epochs(n, trn_size=100):
    # input must be exact multiples of batch_size, for stateful
    # multiples = len(sentences)//batch_size*batch_size
    # sentences_truncated = sentences[:multiples]
    # next_chars_truncated = next_chars[:multiples]
    next_chars_adjusted = np.expand_dims(next_chars,-1)
    trn_size = batch_size * trn_size
    rand_trn_idx = np.random.randint(0, high=len(text)-1-trn_size)
    
    sentences_selected = sentences[rand_trn_idx:rand_trn_idx+trn_size]
    next_chars_selected = next_chars_adjusted[rand_trn_idx:rand_trn_idx+trn_size]
    
    for i in range(n):
        rand_trn_idx = np.random.randint(0, high=len(text)-1-trn_size)
    
        sentences_selected = sentences[rand_trn_idx:rand_trn_idx+trn_size]
        next_chars_selected = next_chars_adjusted[rand_trn_idx:rand_trn_idx+trn_size]
        # model.reset_states()
        h = model.fit(sentences_selected, next_chars_selected, batch_size=batch_size, nb_epoch=1, shuffle=True)
        print(h.history, h.history['loss'])
        weight_file_name = './data/text-generation-lgqm-ver-1-char-cnn-'+str(time.time())+'.h5'
    model.save_weights(weight_file_name)
    print('saved weights: ', weight_file_name)

In [9]:
# NOTE: when cr is not together with lf, a line is biten! But, I doubt it ever happens in original text
# Better learn more to let it avoid naturally

def print_example_inline_seed_non_stateful():
    pre_string = u"第65535节 "
    
    rand_idx = np.random.randint(0, high=len(text)-maxlen-1)
    seed_string = text[rand_idx:rand_idx+maxlen]
    
    for i in range(maxlen+1):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    seed_string = pre_string+seed_string[maxlen:]
    print(seed_string)
    for i in range(3000):
        x=np.array([char2idx[c] for c in seed_string[-maxlen:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(charlist, p=preds)
        seed_string = seed_string + next_char
    print('final: ', seed_string)
    return seed_string

In [10]:
run_epochs(1)

Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.8163    
{'loss': [4.8163385391235352]} [4.8163385391235352]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497651144.43.h5


In [16]:
# gen_text = print_example_inline_seed_non_stateful()
f = codecs.open('./data/generated_text/generated_text_0.txt', encoding='utf-8', mode='w+')
f.write(gen_text)

In [17]:
run_epochs(10)

Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.4430    
{'loss': [4.4429683184623716]} [4.4429683184623716]
Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.0540    
{'loss': [4.0539587378501896]} [4.0539587378501896]
Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.2122    
{'loss': [4.2121559667587283]} [4.2121559667587283]
Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.3015    
{'loss': [4.3014821171760556]} [4.3014821171760556]
Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.3117    
{'loss': [4.3116943573951723]} [4.3116943573951723]
Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.1374    
{'loss': [4.1373786377906798]} [4.1373786377906798]
Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.0418    
{'loss': [4.0417818546295168]} [4.0417818546295168]
Epoch 1/1
6400/6400 [==============================] - 30s - loss: 4.0581    
{'loss': [4.

In [21]:
def generate_text_n(n):
    for i in range(n):
        gen_text = print_example_inline_seed_non_stateful()
        f = codecs.open('./data/generated_text/generated_text_'+str(time.time())+'.txt', encoding='utf-8', mode='w+')
        f.write(gen_text)

In [22]:
run_epochs(1, trn_size=1000)

Epoch 1/1
64000/64000 [==============================] - 304s - loss: 3.8822   
{'loss': [3.8822375457286835]} [3.8822375457286835]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497652204.4.h5


In [23]:
for i in range(10):
    run_epochs(4, trn_size=300)
    generate_text_n(20)

Epoch 1/1
19200/19200 [==============================] - 91s - loss: 3.8823    
{'loss': [3.8823199645678201]} [3.8823199645678201]
Epoch 1/1
19200/19200 [==============================] - 91s - loss: 3.9074    
{'loss': [3.907365799744924]} [3.907365799744924]
Epoch 1/1
19200/19200 [==============================] - 90s - loss: 4.1485    
{'loss': [4.1484501139322916]} [4.1484501139322916]
Epoch 1/1
19200/19200 [==============================] - 91s - loss: 3.9121    
{'loss': [3.9120946542421975]} [3.9120946542421975]
saved weights:  ./data/text-generation-lgqm-ver-1-char-cnn-1497652641.96.h5
第65535节 ，但是村里的官府开始不要了干远并不没见过想来的新身倒装得进行，很大不大，看村的旱灾秀不便死。掌柜两
final:  第65535节 ，但是村里的官府开始不要了干远并不没见过想来的新身倒装得进行，很大不大，看村的旱灾秀不便死。掌柜两声现在是做出来。他估摸着刘知府和中寸买梢叶了。大庆媳妇又没法吃齐。等白子的开销白灰就造成，一想到他家的余地上，都忙种了，更正给蚕种一年春成饭，特别要勒没一份，立刻开始空前，虽然的气氛的不免做零次一直又快客气，就要说是天面官的宾务三万斤，只能和他分容易的村里一想过不来，所谓自己又有一般，沈大家都就是要借究斗有很多，晓得直奔来熬说不登船。一口气刘梦谦在蚕花，光边觉得也是啥公好，连办理不过了。

    去人穿着在杭州城窗、纱灯不让张溥喘船直奔开了。不过是从这乡亲才差出现，两次这笔东西，消息他也甜，还没好宣表的隐患，一干人都不起得可以生丝。办得人家因

KeyboardInterrupt: 